# Core

> The building blocks to the UI

## Imports

In [ ]:
#| default_exp core

In [ ]:
#| hide
#| export
import fasthtml.common as fh
from fh_frankenui.foundations import *
from fasthtml.common import is_listy, Div, P, Span, Script, FastHTML, FT, to_xml, show
from fasthtml.svg import Svg
from enum import Enum, auto
from fasthtml.components import Uk_select,Uk_input_tag
from functools import partial
from itertools import zip_longest
from typing import Union, Tuple, Optional
from fastcore.all import *
import copy

In [ ]:
%%html
<script src="https://cdn.tailwindcss.com"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"></script>
<script type="module" src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js"></script>
<link rel="stylesheet" href="https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css">


<style>
#notebook-container { max-width: none; }
.output_html * { list-style-type: none !important; }
</style>

In [ ]:
#| hide
from functools import partial
from IPython.display import HTML, display, Markdown

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
def enum_to_markdown_table(enum_class):
    headers = ["Option", "Value"]
    rows = [[name, value.value] for name, value in enum_class.__members__.items()]
    
    max_name_len = max(len(row[0]) for row in rows)
    max_value_len = max(len(row[1]) for row in rows)
    
    header = f"| {'Option':<{max_name_len}} | {'Value':<{max_value_len}} |"
    separator = f"|{'-'*(max_name_len+2)}|{'-'*(max_value_len+2)}|"
    body = "\n".join(f"| {row[0]:<{max_name_len}} | {row[1]:<{max_value_len}} |" for row in rows)
    
    return f"{header}\n{separator}\n{body}"

## Theme / Headers

You can select a theme color to and get all the headers.  This brings in the most standard grouping:
+ Tailwind: Optional, but typically FrankenUI is used in conjunction with tailwind
+ Uikit: Many things are styled with uikit
+ UIkit icons: Gives many icons OOTB, and is nice to have handy.
+ Franken WC: Neccesary for some of the (minimal) interactivity components have

In [ ]:
#| export
class Theme(Enum):
    slate = "slate"
    stone = "stone"
    gray = "gray"
    neutral = "neutral"
    red = "red"
    rose = "rose"
    orange = "orange"
    green = "green"
    blue = "blue"
    yellow = "yellow"
    violet = "violet"
    zinc = "zinc"

    def headers(self):
        js = (fh.Script(src="https://cdn.tailwindcss.com"),
              fh.Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"),
              fh.Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"),
              fh.Script(type="module", src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js")
              )
        _url = f"https://unpkg.com/franken-wc@0.0.6/dist/css/{self.value}.min.css"
        return (*js, fh.Link(rel="stylesheet", href=_url))

In [ ]:
hdrs = Theme.blue.headers()
app = FastHTML(hdrs=hdrs)

In [ ]:
#| hide
#| eval: false

from fasthtml.jupyter import *
from uuid import uuid4

if is_port_free(8000):
    app, rt = fh.fast_app(pico=False, hdrs=Theme.blue.headers())
    server = JupyUvi(app)
        
def HShow(comp, app):
    @app.get('/')
    def get(): return comp

    display(HTML(f'<a href="http://localhost:8000/" target="_blank">Open in new tab</a>'))
    return HTMX("/")

Show = partial(HShow, app=app)

## Text Style

In [ ]:
#| export
class TextT(VEnum):
    'Text Styles from https://franken-ui.dev/docs/text'
    def _generate_next_value_(name, start, count, last_values):
        return str2ukcls('text', name)
    
    # Text Style
    lead,meta, italic = auto(), auto(), auto()
    # Text Size
    small, default, large = auto(), 'uk-text', auto()
    # Text Weight
    light, normal, bold, lighter, bolder = auto(),auto(),auto(),auto(),auto()
    # Text Transform
    capitalize,uppercase, lowercase = auto(),auto(),auto()
    # Text Decoration
    decoration_none = auto()
    # Text Color
    muted,primary,secondary, success,warning, danger = auto(),auto(),auto(),auto(),auto(),auto()
    # Text Alignment
    left, right,center,justify = auto(), auto(), auto(), auto()
    # Vertical Alignment
    top,middle,bottom, baseline = auto(),auto(),auto(),auto()
    
    # Text Wrapping
    truncate,break_,nowrap = auto(),auto(),auto()

In [ ]:
print(enum_to_markdown_table(TextT))

| Option          | Value                   |
|-----------------|-------------------------|
| lead            | uk-text-lead            |
| meta            | uk-text-meta            |
| italic          | uk-text-italic          |
| small           | uk-text-small           |
| default         | uk-text                 |
| large           | uk-text-large           |
| light           | uk-text-light           |
| normal          | uk-text-normal          |
| bold            | uk-text-bold            |
| lighter         | uk-text-lighter         |
| bolder          | uk-text-bolder          |
| capitalize      | uk-text-capitalize      |
| uppercase       | uk-text-uppercase       |
| lowercase       | uk-text-lowercase       |
| decoration_none | uk-text-decoration-none |
| muted           | uk-text-muted           |
| primary         | uk-text-primary         |
| secondary       | uk-text-secondary       |
| success         | uk-text-success         |
| warning         | uk-text-warnin

In [ ]:
#| export
class TextFont(Enum):
    "Combinations of TextT that are particularly useful"
    def __add__(self, other):   return stringify((self, other))
    def __radd__(self, other):  return stringify((other, self)) 
    def __str__(self): return self.value
    muted_sm = stringify((TextT.muted, TextT.small))
    muted_lg = stringify((TextT.muted, TextT.large))
    bold_sm = stringify((TextT.bold, TextT.small))
    

## Basic Components

In [ ]:
#| export
def Alert(*c, cls=(), **kwargs):
    return Div(*c, cls=('uk-alert', stringify(cls)), uk_alert=True, **kwargs)

In [ ]:
Alert()

```html
<div class="uk-alert " uk-alert></div>

```

## Button

In [ ]:
#| export
class ButtonT(VEnum):
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('button', name)
    default = auto()
    primary = auto()
    secondary = auto()
    danger = auto()
    text = auto()
    link = auto()
    ghost = auto()

In [ ]:
Markdown(enum_to_markdown_table(ButtonT))

| Option    | Value               |
|-----------|---------------------|
| default   | uk-button-default   |
| primary   | uk-button-primary   |
| secondary | uk-button-secondary |
| danger    | uk-button-danger    |
| text      | uk-button-text      |
| link      | uk-button-link      |
| ghost     | uk-button-ghost     |

In [ ]:
#| export
def Button(*c:str|FT,                     # Components to go inside the Button
           cls:str|Enum=ButtonT.default,  # cls for the Button (see ButtonT for style options)
           **kwargs                       # any other kwargs will be passed to the button 
          )-> FT:                         # Button w/ `type=button` and `uk-button` cls
    "A Button with Uk Styling"
    return fh.Button(*c, cls=('uk-button',stringify(cls)), type='button', **kwargs)

In [ ]:
Button("Click Me")

```html
<button type="button" class="uk-button uk-button-default">Click Me</button>
```

In [ ]:
show(Div(Button("Default Class"),
        Button("Primary Class", cls=ButtonT.primary),
        Button("Secondary Class", cls=ButtonT.secondary),
        Button("Danger Class", cls=ButtonT.danger),
        Button("Ghost Class", cls=ButtonT.ghost),
        Button("Link Class", cls=ButtonT.link),
        Button("Text Class", cls=ButtonT.text),))

## Headings

In [ ]:
#| export
def H1(*c:FT|str,       # Components to go inside the Heading
       cls:Enum|str|tuple=(),   # cls for the Heading
       **kwargs  # any other kwargs will be passed to the Heading
      )->FT: # Heading with `class=uk-h1` cls
    "A H1 with Uk Styling"
    return fh.H1(*c, cls=('uk-h1',stringify(cls)), **kwargs)

In [ ]:
#| export
def H2(*c:FT|str,       # Components to go inside the Heading
       cls:Enum|str|tuple=(),   # cls for the Heading
       **kwargs  # any other kwargs will be passed to the Heading
      )->FT: # Heading with `class=uk-h2` cls
    "A H2 with Uk Styling"
    return fh.H2(*c, cls=('uk-h2',stringify(cls)), **kwargs)

In [ ]:
#| export
def H3(*c:FT|str,       # Components to go inside the Heading
       cls:Enum|str|tuple=(),   # cls for the Heading
       **kwargs  # any other kwargs will be passed to the Heading
      )->FT: # Heading with `class=uk-h3` cls
    "A H3 with Uk Styling"
    return fh.H3(*c, cls=('uk-h3',stringify(cls)), **kwargs)



In [ ]:
#| export
def H4(*c:FT|str,       # Components to go inside the Heading
       cls:Enum|str|tuple=(),   # cls for the Heading
       **kwargs  # any other kwargs will be passed to the Heading
      )->FT: # Heading with `class=uk-h4` cls
    "A H4 with Uk Styling"
    return fh.H4(*c, cls=('uk-h4',stringify(cls)), **kwargs)


In [ ]:
show(Div(
    H1("Heading 1 (H1)"),
    H2("Heading 2 (H2)"),
    H3("Heading 3 (H3)"),
    H4("Heading 4 (H4)")))

## Dividers

In [ ]:
#| export
def UkHSplit(*c, cls=(), line_cls=(), text_cls=()):
    # Divider FrankenUI stuff
    cls, line_cls, text_cls = map(stringify,(cls, line_cls, text_cls))
    return Div(cls='relative ' + cls)(
        Div(cls="absolute inset-0 flex items-center " + line_cls)(Span(cls="w-full border-t border-border")),
        Div(cls="relative flex justify-center " + text_cls)(Span(cls="bg-background px-2 ")(*c)))

In [ ]:
UkHSplit()

```html
<div class="relative ">
  <div class="absolute inset-0 flex items-center ">
<span class="w-full border-t border-border"></span>  </div>
  <div class="relative flex justify-center ">
<span class="bg-background px-2 "></span>  </div>
</div>

```

In [ ]:
#| export
def UkHLine(lwidth=2, y_space=4): return Div(cls=f"my-{y_space} h-[{lwidth}px] w-full bg-secondary")

In [ ]:
UkHLine()

```html
<div class="my-4 h-[2px] w-full bg-secondary"></div>

```

In [ ]:
#| export
def UkNavDivider(): return fh.Li(cls="uk-nav-divider")

In [ ]:
UkNavDivider()

```html
<li class="uk-nav-divider"></li>

```

## Articles & Containers & Sections

In [ ]:
#| export
def Article(*c, cls=(), **kwargs):
    return fh.Article(*c, cls=('uk-article',stringify(cls)), **kwargs)

def ArticleTitle(*c, cls=(), **kwargs):
    return H1(*c, cls=('uk-article-title',stringify(cls)), **kwargs)

def ArticleMeta(*c, cls=(), **kwargs):
    return P(*c, cls=('uk-article-meta',stringify(cls)), **kwargs)

In [ ]:
show(Article(ArticleTitle("Article Title"), ArticleMeta("By: John Doe")))

In [ ]:
#| export
class ContainerT(VEnum):
    'Max width container sizes from https://franken-ui.dev/docs/container'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('container', name)
    xsmall = auto()
    small = auto()
    large = auto()
    xlarge = auto()
    expand = auto()

In [ ]:
#| export
def Container(*c, cls=(), **kwargs): 
    return Div(*c, cls=('uk-container',stringify(cls)), **kwargs)

In [ ]:
Container(cls=ContainerT.xsmall)

```html
<div class="uk-container uk-container-xsmall"></div>

```

In [ ]:
# Add background colors to the containers
# show(Div(Container(cls=(ContainerT.xsmall, "bg-blue-200"))("XSmall container"),
#          Container(cls=(ContainerT.small, "bg-red-200"))("Small container"),
#          Container(cls=(ContainerT.large, "bg-yellow-200"))("Large container")))

In [ ]:
#| export
class SectionT(VEnum):
    'Section styles from UIkit'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('section', name)
    default = auto()
    muted = auto()
    primary = auto()
    secondary = auto()
    xsmall = auto()
    small = auto()
    large = auto()
    xlarge = auto()
    remove_vertical = auto()

In [ ]:
print(enum_to_markdown_table(SectionT))

| Option          | Value                      |
|-----------------|----------------------------|
| default         | uk-section-default         |
| muted           | uk-section-muted           |
| primary         | uk-section-primary         |
| secondary       | uk-section-secondary       |
| xsmall          | uk-section-xsmall          |
| small           | uk-section-small           |
| large           | uk-section-large           |
| xlarge          | uk-section-xlarge          |
| remove_vertical | uk-section-remove-vertical |


In [ ]:
#| export
def Section(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-section',stringify(cls)), **kwargs)

## Forms & Inputs

In [ ]:
#|export
def Fieldset(*c, cls=(), **kwargs): 
    return fh.Fieldset(*c, cls=('uk-fieldset',stringify(cls)), **kwargs)

def Legend(*c, cls=(), **kwargs): 
    return fh.Legend(*c, cls=('uk-legend',stringify(cls)), **kwargs)

In [ ]:
#| export
def Input(*c, cls=(), **kwargs):               return fh.Input(*c, cls=('uk-input',stringify(cls)), **kwargs)
def Select(*c, cls=(), **kwargs):              return fh.Select(*c, cls=('uk-select',stringify(cls)), **kwargs)
def Radio(*c, cls=(), **kwargs):               return fh.Input(*c, cls=('uk-radio',stringify(cls)), type='radio', **kwargs)
def CheckboxX(*c, cls=(), **kwargs):           return fh.Input(*c, cls=('uk-checkbox',stringify(cls)), type='checkbox', **kwargs)
def Range(*c, cls=(), **kwargs):               return fh.Input(*c, cls=('uk-range',stringify(cls)), type='range', **kwargs)
def Toggle_switch(*c, cls=(), **kwargs):       return fh.Input(*c, cls=('uk-toggle-switch',stringify(cls)), type='checkbox', **kwargs)
def TextArea(*c, cls=(), **kwargs):            return fh.Textarea(*c, cls=('uk-textarea',stringify(cls)), **kwargs)
def Button(*c, cls=ButtonT.default,  **kwargs):return fh.Button(*c, cls=('uk-button',stringify(cls)), type='button', **kwargs)
def Switch(*c, cls=(), **kwargs):              return fh.Input(*c, cls=('uk-toggle-switch',stringify(cls)), type='checkbox', **kwargs)

In [ ]:
show(Div(
    P("Input: ", Input()),
    P("Select: ", Select(fh.Option("Option 1"), fh.Option("Option 2"))),
    P("Radio: ", Radio()),
    P("Checkbox: ", CheckboxX()),
    P("Range: ", Range()),
    P("Toggle switch: ", Toggle_switch()),
    P("TextArea: ", TextArea()),
    P("Button: ", Button("Click me")),
    P("Switch: ", Switch()),
    cls="space-y-4"
))

In [ ]:
#| export
class LabelT(VEnum):
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('label', name)
    primary = auto()
    secondary = auto()
    danger = auto()

In [ ]:
#| export
def Label(*c, cls=(), **kwargs):
    return fh.Label(*c, cls=('uk-label',stringify(cls)), **kwargs)

In [ ]:
show(Label("My Label"))

In [ ]:
#|export
def FormLabel(*c, cls=(), **kwargs):
    return fh.Label(*c, cls=('uk-form-label',stringify(cls)), **kwargs)

In [ ]:
show(FormLabel('My Form Label'))

In [ ]:
#| export
def UkFormSection(title, description, *c, button_txt='Update', outer_margin=6, inner_margin=6):
    return Div(cls=f'space-y-{inner_margin} py-{outer_margin}')(
        Div(H3(title), P(description, cls=TextFont.muted_sm)),
        UkHSplit(), *c,
        Div(Button(button_txt, cls=ButtonT.primary)) if button_txt else None)

In [ ]:
UkFormSection('a','b','c')

```html
<div class="space-y-6 py-6">
  <div>
    <h3 class="uk-h3 ">a</h3>
    <p class="uk-text-muted uk-text-small">b</p>
  </div>
  <div class="relative ">
    <div class="absolute inset-0 flex items-center ">
<span class="w-full border-t border-border"></span>    </div>
    <div class="relative flex justify-center ">
<span class="bg-background px-2 "></span>    </div>
  </div>
c  <div>
<button type="button" class="uk-button uk-button-primary">Update</button>  </div>
</div>

```

## Links

In [ ]:
#| export
class LinkT(VEnum):
    'Link styles from https://franken-ui.dev/docs/link'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('link', name)
    muted = auto()
    text = auto()
    reset = auto()

In [ ]:
#| export
def Link(*c, cls=(), **kwargs):  
    return fh.A(*c, cls=('uk-link',stringify(cls)), **kwargs)

In [ ]:
show(Div(cls='space-x-4')(Link('Default Link'),
                     Link('Muted Link', cls=LinkT.muted),
                     Link('Text Link', cls=LinkT.text),
                     Link('Reset Link', cls=LinkT.reset)))

## Lists

In [ ]:
#| export
class ListT(VEnum):
    'List styles from https://franken-ui.dev/docs/list'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('list', name)
    disc = auto()
    circle = auto()
    square = auto()
    decimal = auto()
    hyphen = auto()
    muted = auto()
    primary = auto()
    secondary = auto()
    bullet = auto()
    divider = auto()
    striped = auto()

In [ ]:
#| export
def List(*c, cls=(), **kwargs): return fh.Ul(*c, cls=('uk-list',stringify(cls)), **kwargs)

In [ ]:
list_options = [(style,str(cls)) for style,cls in ListT.__members__.items()]
def create_list(style, cls): return List(fh.Li("Item 1"), fh.Li("Item 2"), cls=cls)

lists = [Div(H4(f"{style} List:"), create_list(style, cls)) for style, cls in list_options]
show(Div(*lists, cls="grid grid-cols-4"))

## Modal

In [ ]:
#| export
def ModalContainer(*c, cls=(), **kwargs):   return fh.Div(*c, cls=('uk-modal-container',stringify(cls)), uk_modal=True, **kwargs)
def ModalDialog(*c, cls=(), **kwargs):      return fh.Div(*c, cls=('uk-modal-dialog',   stringify(cls)),                **kwargs)
def ModalHeader(*c, cls=(), **kwargs):      return fh.Div(*c, cls=('uk-modal-header',   stringify(cls)),                **kwargs)
def ModalBody(*c, cls=(), **kwargs):        return fh.Div(*c, cls=('uk-modal-body',     stringify(cls)),                **kwargs)
def ModalFooter(*c, cls=(), **kwargs):      return fh.Div(*c, cls=('uk-modal-footer',   stringify(cls)),                **kwargs)
def ModalTitle(*c, cls=(), **kwargs):       return fh.H2(*c,  cls=('uk-modal-title',    stringify(cls)),                **kwargs)
def ModalCloseButton(*c, cls=(), **kwargs): return Button(*c, cls=('uk-modal-close',    stringify(cls)),                **kwargs)

In [ ]:
show(ModalCloseButton('Modal Close Button', cls=ButtonT.primary))

In [ ]:
#| export
def Modal(*c,
        header=None,          # Components that go in the header
        footer=None,          # Components that go in the footer
        cls=(),               # class for outermost container
        dialog_cls=(),        # classes for the dialog
        header_cls='p-6',     # classes for the header
        body_cls='space-y-6', # classes for the body
        footer_cls=(),        # classes for the footer
        id='',                # id for the outermost container
        **kwargs              # classes for the outermost container
        ): # Modal
    "Create a Modal using the appropriate Modal* classes to put the boilerplate in the appropriate places for you"
    cls, dialog_cls, header_cls, body_cls, footer_cls = map(stringify, (cls, dialog_cls, header_cls, body_cls, footer_cls))
    res = []
    if header: res.append(ModalHeader(cls=header_cls)(header))
    res.append(ModalBody(cls=body_cls)(*c))
    if footer: res.append(ModalFooter(cls=footer_cls)(footer))
    return ModalContainer(ModalDialog(*res, cls=dialog_cls), cls=cls, id=id, **kwargs)

## Navs

## Padding | Positioning

In [ ]:
#| export
class PaddingT(VEnum):
    'Padding Modifiers from https://franken-ui.dev/docs/padding'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('padding', name)
    xsmall = auto()
    small = auto()
    default = ''
    medium = auto()
    large = auto()
    xlarge = auto()
    remove = auto()
    remove_top = auto()
    remove_bottom = auto()
    remove_left = auto()
    remove_right = auto()
    remove_vertical = auto()
    remove_horizontal = auto()

In [ ]:
print(enum_to_markdown_table(PaddingT))

| Option            | Value                        |
|-------------------|------------------------------|
| xsmall            | uk-padding-xsmall            |
| small             | uk-padding-small             |
| default           |                              |
| medium            | uk-padding-medium            |
| large             | uk-padding-large             |
| xlarge            | uk-padding-xlarge            |
| remove            | uk-padding-remove            |
| remove_top        | uk-padding-remove-top        |
| remove_bottom     | uk-padding-remove-bottom     |
| remove_left       | uk-padding-remove-left       |
| remove_right      | uk-padding-remove-right      |
| remove_vertical   | uk-padding-remove-vertical   |
| remove_horizontal | uk-padding-remove-horizontal |


In [ ]:
#| export
class PositionT(VEnum):
    'Position modifiers from https://franken-ui.dev/docs/position'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('position', name)
    top = auto()
    bottom = auto()
    left = auto()
    right = auto()
    top_left = auto()
    top_center = auto()
    top_right = auto()
    center = auto()
    center_left = auto()
    center_right = auto()
    bottom_left = auto()
    bottom_center = auto()
    bottom_right = auto()
    center_horizontal = auto()
    center_vertical = auto()

In [ ]:
print(enum_to_markdown_table(PositionT))

| Option            | Value                         |
|-------------------|-------------------------------|
| top               | uk-position-top               |
| bottom            | uk-position-bottom            |
| left              | uk-position-left              |
| right             | uk-position-right             |
| top_left          | uk-position-top-left          |
| top_center        | uk-position-top-center        |
| top_right         | uk-position-top-right         |
| center            | uk-position-center            |
| center_left       | uk-position-center-left       |
| center_right      | uk-position-center-right      |
| bottom_left       | uk-position-bottom-left       |
| bottom_center     | uk-position-bottom-center     |
| bottom_right      | uk-position-bottom-right      |
| center_horizontal | uk-position-center-horizontal |
| center_vertical   | uk-position-center-vertical   |


## Other

In [ ]:
#| export
def Placeholder(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-placeholder',stringify(cls)), **kwargs)

In [ ]:
#| export
def Progress(*c, cls=(), value="", max="", **kwargs):
    return fh.Progress(*c, value=value, max=max, cls=('uk-progress',stringify(cls)), **kwargs)

In [ ]:
#| export
def Sticky(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=stringify(cls), uk_sticky=True, **kwargs)

## Icons and Avatars

In [ ]:
#| export
def UkIcon(icon,    # Icon name from https://getuikit.com/docs/icon
           ratio=1, # Icon ratio/size 
           cls=()   # Span classes
          ):        # Span with Icon
    "Creates a Span with the given icon"
    return Span(uk_icon=f"icon: {icon}; ratio: {ratio}",cls=stringify(cls))

In [ ]:
UkIcon('heart', 0.5)

```html
<span uk-icon="icon: heart; ratio: 0.5"></span>
```

In [ ]:
#| export
# def Img(*args, data_src="", cls=(), **kwargs):
#     return fh.Div, *args, data_src=data_src, uk_img=True, cls=stringify(cls), **kwargs)

In [ ]:
#| export
def DiceBearAvatar(seed_name, # Seed name (ie 'Isaac Flath')
                   h,         # Height 
                   w          # Width
                  ):          # Span with Avatar
    url = 'https://api.dicebear.com/8.x/lorelei/svg?seed='
    return Span(cls=f"relative flex h-{h} w-{w} shrink-0 overflow-hidden rounded-full bg-accent")(
            fh.Img(cls=f"aspect-square h-{h} w-{w}", alt="Avatar", src=f"{url}{seed_name}"))

In [ ]:
show(DiceBearAvatar('Isaac Flath', 14, 14))

In [ ]:
#| export
def UkIconButton(*c, sz='small', cls=(), **kwargs):
    "Creates an `IconButton` with uk styling"
    if sz not in ('small','medium','large'): raise ValueError(f"Invalid size '{sz}'. Must be 'small', 'medium', or 'large'.")
    return Button(cls=f'uk-icon-button uk-icon-button-{sz} ' + stringify(cls), **kwargs)(*c)

In [ ]:
UkIconButton(UkIcon('home'))

```html
<button type="button" class="uk-button uk-icon-button uk-icon-button-small "><span uk-icon="icon: home; ratio: 1"></span></button>
```

## Flexbox | Grid

The most common pattern for spacing it to organize the general high level page layout with a `Grid`, and smaller components with `Flex`.

:::{.callout-tip}
Play [Flex Box Froggy](https://flexboxfroggy.com/) to get an understanding of flex box.
:::

In [ ]:
#| export
class FlexT(VEnum):
    'Flexbox modifiers from UIkit'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('flex', name)
    
    # Display
    block, inline = 'uk-flex', auto()
    # Horizontal Alignment
    left, center, right, between, around = auto(), auto(), auto(), auto(), auto()
    # Vertical Alignment
    stretch, top, middle, bottom = auto(), auto(), auto(), auto()
    # Direction
    row, row_reverse, column, column_reverse = auto(), auto(), auto(), auto()
    # Wrap
    nowrap, wrap, wrap_reverse = auto(), auto(), auto()

In [ ]:
print(enum_to_markdown_table(FlexT))

| Option         | Value                  |
|----------------|------------------------|
| block          | uk-flex                |
| inline         | uk-flex-inline         |
| left           | uk-flex-left           |
| center         | uk-flex-center         |
| right          | uk-flex-right          |
| between        | uk-flex-between        |
| around         | uk-flex-around         |
| stretch        | uk-flex-stretch        |
| top            | uk-flex-top            |
| middle         | uk-flex-middle         |
| bottom         | uk-flex-bottom         |
| row            | uk-flex-row            |
| row_reverse    | uk-flex-row-reverse    |
| column         | uk-flex-column         |
| column_reverse | uk-flex-column-reverse |
| nowrap         | uk-flex-nowrap         |
| wrap           | uk-flex-wrap           |
| wrap_reverse   | uk-flex-wrap-reverse   |


In [ ]:
Div(cls=(FlexT.block,FlexT.left,FlexT.middle,'some_cls'))

```html
<div class="uk-flex uk-flex-left uk-flex-middle some_cls"></div>

```

In [ ]:
#| export
class GridT(VEnum):
    'Grid modifiers from UIkit'
    def _generate_next_value_(name, start, count, last_values):
        return str2ukcls('grid', name)
    
    small, medium, large, collapse = auto(), auto(), auto(), auto()

In [ ]:
print(enum_to_markdown_table(GridT))

| Option   | Value            |
|----------|------------------|
| small    | uk-grid-small    |
| medium   | uk-grid-medium   |
| large    | uk-grid-large    |
| collapse | uk-grid-collapse |


In [ ]:
#|export
def Grid(*div,      # Divs/Containers that should be divided into a grid
         cols=None,  # Number of columns (defaults to min(len(div),5))
         cls=GridT.small,  # Additional classes for Grid Div
         **kwargs # Additional args for Grid Div
        ):
    """Creates a grid with the given number of columns, often used for a grid of cards"""
    cols = cols if cols else min(len(div),5)
    cls = stringify(cls)
    return Div(cls=(f'grid grid-cols-{cols}',cls), **kwargs)(*div)

In [ ]:
Grid(*map(Div, range(6)),cols=3, cls=GridT.small)

```html
<div class="grid grid-cols-3 uk-grid-small">
  <div>0</div>
  <div>1</div>
  <div>2</div>
  <div>3</div>
  <div>4</div>
  <div>5</div>
</div>

```

In [ ]:
#| export
def ResponsiveGrid(*c, sm=1, md=2, lg=3, xl=4, gap=2, cls='', **kwargs):
    "Creates a responsive grid with the given number of columns for different screen sizes"
    return Div(cls=f'grid grid-cols-{sm} md:grid-cols-{md} lg:grid-cols-{lg} xl:grid-cols-{xl} gap-{gap} ' + stringify(cls), **kwargs)(*c)

We can use the `FlexT` enum above to create `Div`s that arrange components in a flex box in different ways.  

For example, `FullySpacedDiv` applies a very common pattern so we've made a function that helps to this for you.  But if you look at the code, it's really just using the enum to apply the right combonation of flex classes.

In [ ]:
#| exports
def FullySpacedDiv(*c,                # Components
                   cls='uk-width-1-1',# Classes for outer div
                   **kwargs           # Additional args for outer div
                  ):                  # Div with spaced components via flex classes
    "Creates a flex div with it's components having as much space between them as possible"
    cls = stringify(cls)
    return Div(cls=(FlexT.block,FlexT.between,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
FullySpacedDiv(*map(P, range(3)))

```html
<div class="uk-flex uk-flex-between uk-flex-middle uk-width-1-1">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def CenteredDiv(*c,      # Components
                cls=(),  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components centered in it
    "Creates a flex div with it's components centered in it"
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.column,FlexT.middle,FlexT.center,cls),**kwargs)(*c)

In [ ]:
CenteredDiv(*map(P, range(3)))

```html
<div class="uk-flex uk-flex-column uk-flex-middle uk-flex-center ">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def LAlignedDiv(*c,      # Components
                cls=(),  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components aligned to the left
    "Creates a flex div with it's components aligned to the left"
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.left,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
LAlignedDiv(*map(P, range(3)))

```html
<div class="uk-flex uk-flex-left uk-flex-middle ">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def RAlignedDiv(*c,      # Components
                cls=(),  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components aligned to the right
    "Creates a flex div with it's components aligned to the right"
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.right,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
RAlignedDiv(*map(P, range(3)))

```html
<div class="uk-flex uk-flex-right uk-flex-middle ">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def VStackedDiv(*c, cls='', **kwargs):
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.col,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
#| export
def HStackedDiv(*c, cls='', **kwargs):
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.row,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
#| export
def SpaceBetweenDiv(*c, cls='', **kwargs):
    cls = stringify(cls)
    return Div(cls=(FlexT.block,FlexT.between,FlexT.middle,cls), **kwargs)(*c)

## Labeled Inputs

Inputs of various types often go with a label.  Because of this we created functions to do this for you along with properly linking the `for` attribute from the lable to the input.  We also have some nice defaults, such as putting a little spacing between the label and the input

In [ ]:
#| export
def GenericLabelInput(
               label:str|FT,
               lbl_cls='',
               input_cls='',
               container=Div, 
               container_cls='',
               id='',
               input_fn=noop, 
                **kwargs
                ):
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UkTextArea` which is created for you in this library"
    if isinstance(label, str) or label.tag != 'label': 
        label = FormLabel(cls=stringify(lbl_cls), fr=id)(label)
    inp = input_fn(id=id, cls=stringify(input_cls), **kwargs)        
    if container: return container(label, inp, cls=stringify(container_cls))
    return label, inp

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelInput(*args, **kwargs): return GenericLabelInput(*args, input_fn=Input, **kwargs)

In [ ]:
#| export
def LabelRadio(label:str|FT,
               lbl_cls='',
               input_cls='',
               container=Div, 
               container_cls='',
               id='',
                **kwargs
                ):
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UkTextArea` which is created for you in this library"
    if isinstance(label, str) or label.tag != 'label': 
        label = FormLabel(cls=stringify(lbl_cls), fr=id)(label)
    inp = Radio(id=id, cls=stringify(input_cls), **kwargs)        
    if container: return container(inp, label, cls=stringify(container_cls))
    return inp, label

In [ ]:
fh.show(LabelRadio('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelCheckboxX(*args, **kwargs): return GenericLabelInput(*args, input_fn=CheckboxX, **kwargs)

In [ ]:
fh.show(LabelCheckboxX('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelRange(*args, **kwargs): return GenericLabelInput(*args, input_fn=Range, **kwargs)

In [ ]:
fh.show(LabelRange('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelToggle_switch(*args, **kwargs): return GenericLabelInput(*args, input_fn=Toggle_switch, **kwargs)

In [ ]:
fh.show(Toggle_switch('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelTextArea(*args, **kwargs): return GenericLabelInput(*args, input_fn=TextArea, **kwargs)

In [ ]:
fh.show(LabelTextArea('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn'])
def LabelSwitch(*args, **kwargs): return GenericLabelInput(*args, input_fn=Switch, **kwargs)

In [ ]:
fh.show(LabelSwitch('mylabel',container=Form))

In [ ]:
#| export
def LabelSelect(*option,
               label:str|FT,
               lbl_cls='',
               input_cls='',
               container=Div, 
               container_cls='',
               id='',
                **kwargs
                ):
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UkTextArea` which is created for you in this library"
    if isinstance(label, str) or label.tag != 'label': 
        label = FormLabel(lbl_cls=stringify(lbl_cls), fr=id)(label)
    inp = Select(*option, id=id, cls=stringify(input_cls), **kwargs)        
    if container: return container(label, inp, cls=stringify(container_cls))
    return label, inp

In [ ]:
fh.show(LabelSelect(Option(1),Option(2),label='mylabel',container=Form))

## Select

In [ ]:
#| export
def Options(*c,                    # Content for an `Option`
            selected_idx:int=None, # Index location of selected `Option`
            disabled_idxs:set=None # Idex locations of disabled `Options`
           ):
    "Helper function to wrap things into `Option`s for use in `UkSelect`"
    return [fh.Option(o,selected=i==selected_idx, disabled=disabled_idxs and i in disabled_idxs) for i,o in enumerate(c)]

In [ ]:
list(map(to_xml,Options('option1','option2','option3', selected_idx=1, disabled_idxs={0,2})))

['<option disabled>option1</option>',
 '<option selected>option2</option>',
 '<option disabled>option3</option>']

In [ ]:
#| export
def UkSelect(*option,            # Options for the select dropdown (can use `Options` helper function to create)
             label=(),           # String or FT component for the label
             lbl_cls=(),         # Additional classes for the label
             inp_cls=(),         # Additional classes for the select input
             cls=('space-y-2',), # Classes for the outer div
             id="",              # ID for the select input
             name="",            # Name attribute for the select input
             placeholder="",     # Placeholder text for the select input
             searchable=False,   # Whether the select should be searchable
             **kwargs):          # Additional arguments passed to Uk_select
    "Creates a select dropdown with uk styling"
    lbl_cls, inp_cls, cls = map(stringify, (lbl_cls, inp_cls, cls))
    if label: 
        lbl = FormLabel(cls=f'{lbl_cls}', fr=id)(label) 
    select = Uk_select(*option, cls=inp_cls, uk_cloak=True, id=id, 
                       name=name, placeholder=placeholder, searchable=searchable, **kwargs)
    return Div(cls=cls)(lbl, select) if label else Div(cls=cls)(select)

In [ ]:
fh.show(Div(cls='space-y-2')(LabelSelect(*Options('a', 'b'), label='MyLabel', placeholder='MyPlaceholder')))

## Nav

A `Nav` is something highly versatile that is the foundation for many things.  Three common places to use a Nav:

    + On its own as a sidebar navigation
    + In a button to give it a drop-down like behavior
    + In a Navbar to give drop down options in the navbar

In [ ]:
#| export
class NavT(VEnum):
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('nav', name)
    default = auto()
    primary = auto()
    secondary = auto()

In [ ]:
#| export
def NavContainer(*li, 
                 cls=NavT.primary,
                 parent=True, 
                 uk_nav=False, #True for default collapsible behavior, see https://franken-ui.dev/docs/nav#component-options for more advanced options
                 **kwargs):
    return fh.Ul(*li, uk_nav=uk_nav, cls=(f"uk-nav{'' if parent else '-sub'}", stringify(cls)),**kwargs)

In [ ]:
mbrs1 = [Li(A('Option 1'), cls='uk-active'), 
         Li(A('Option 2')),
         Li(A('Option 3'))]

In [ ]:
show(NavContainer(*mbrs1))

In [ ]:
#| export
def NavParentLi(*nav_container, cls=(), **kwargs): return fh.Li(*nav_container,  cls=('uk-parent',  stringify(cls)),**kwargs)
def NavDividerLi(*c,cls=(), **kwargs): return fh.Li(*c, cls=('uk-nav-divider', stringify(cls)),**kwargs)
def NavHeaderLi(*c,cls=(), **kwargs): return fh.Li(*c, cls=('uk-nav-header', stringify(cls)),**kwargs)
def NavSubtitle(*c,cls=TextFont.muted_sm, **kwargs): return fh.Div(*c, cls=('uk-nav-subtitle', stringify(cls)),**kwargs)
def NavCloseLi(*c,cls=(), **kwargs): return fh.Li(*c, cls=('uk-drop-close', stringify(cls)),**kwargs)

In [ ]:
NavDividerLi()

```html
<li class="uk-nav-divider "></li>

```

In [ ]:
mbrs2 = [Li(A('Child 1')), Li(A('Child 2')),Li(A('Child 3'))]

show(
    NavContainer(
        NavHeaderLi("NavHeaderLi"),
        *mbrs1,
        Li(A(href='')(Div("Subtitle Ex",NavSubtitle("NavSubtitle text to be shown")))),
        NavDividerLi(),
        NavParentLi(
            A('Parent Name'),
            NavContainer(*mbrs2,parent=False)),
    )
)

In [ ]:
def NavParentIcon(): return Span(uk_nav_parent_icon=True)

## NavBar

In [ ]:
#| export
def NavBarContainer(*c, 
                    cls=(),
                    container_cls=ContainerT.expand,
                    uk_navbar=True,
                    **kwargs): 
    return fh.Div(Container(Div(*c, uk_navbar=uk_navbar),cls=stringify(container_cls)), cls=('uk-navbar-container',stringify(cls)), **kwargs)
def NavBarLSide(*c,  cls=(), **kwargs): return fh.Div(*c, cls=('uk-navbar-left',  stringify(cls)), **kwargs)
def NavBarRSide(*c,  cls=(), **kwargs): return fh.Div(*c, cls=('uk-navbar-right', stringify(cls)), **kwargs)
def NavBarCenter(*c, cls=(), **kwargs): return fh.Div(*c, cls=('uk-navbar-center',stringify(cls)), **kwargs)

In [ ]:
#| export
def NavBarNav(*li, cls=(), **kwargs): return fh.Nav(*li, cls=('uk-navbar-nav',      stringify(cls)),                 **kwargs)

In [ ]:
#| export
def NavBarSubtitle(title, subtitle, cls=(), subtitle_cls=TextFont.muted_sm, **kwargs): 
    return fh.Div(title,fh.Div(subtitle, cls=('uk-navbar-subtitle', stringify(subtitle_cls))), cls=stringify(cls), **kwargs)

In [ ]:
#| export
def NavBarNavContainer(*li, 
                       cls=NavT.primary,
                         parent=True, 
                         uk_nav=False, #True for default collapsible behavior, see https://franken-ui.dev/docs/nav#component-options for more advanced options
                         **kwargs):
    return Div(cls="uk-navbar-dropdown")(NavContainer(*li, cls=('uk-navbar-dropdown-nav z-2147483647',stringify(cls)), uk_nav=uk_nav, parent=parent, **kwargs))

In [ ]:
#|export
def NavBarParentIcon(): return Span(uk_navbar_parent_icon=True)

In [ ]:
l = NavBarNav(
            Li(cls='uk-active')(A("Active",href='')),
            Li(A("Parent",href=''),
                NavBarNavContainer(
                    Li(cls='uk-active')(A("Active",href='')),
                    Li(A("Item",href='')),
                    Li(A("Item",href='')))),
            Li(A("Item",href='')))

r = NavBarNav(
            Li(cls='uk-active')(A(NavBarSubtitle("Title","Subtitle"),href='')),
            Li(
                A("DropDown",NavBarParentIcon(),href=''),
                NavBarNavContainer(
                    NavHeaderLi("NavHeaderLi"),
                    *mbrs1,
                    Li(A(href='')(Div("Subtitle Ex",NavSubtitle("NavSubtitle text to be shown")))),
                    NavDividerLi(),
                    NavParentLi(
                        A('Parent Name'),
                        NavContainer(*mbrs2,parent=False))
                )),
            
    Li(A(NavBarSubtitle("Title","Subtitle"),href='')))

In [ ]:
# Show(
# NavBarContainer(
#     NavBarLSide(l),
#     NavBarRSide(r)
# )
# )

## DropDown

In [ ]:
#| export
def DropDownNavContainer(*li, 
                         cls=NavT.primary,
                         parent=True, 
                         uk_nav=False, #True for default collapsible behavior, see https://franken-ui.dev/docs/nav#component-options for more advanced options
                         **kwargs):
    return Div(cls = 'uk-drop uk-dropdown',uk_dropdown=True)(NavContainer(*li, cls=('uk-dropdown-nav',stringify(cls)), uk_nav=uk_nav, parent=parent, **kwargs))

In [ ]:
# Show(
# Div(Button('test'),
#     DropDownNavContainer(
#         NavHeaderLi("NavHeaderLi"),
#         *mbrs1,
#         Li(A(href='')(Div("Subtitle Ex",NavSubtitle("NavSubtitle text to be shown")))),
#         NavDividerLi(),
#         NavParentLi(
#             A('Parent Name'),
#             NavContainer(*mbrs2,parent=False)),
# ))
# )

## Tabs

In [ ]:
#| export
def TabContainer(*li,cls='', alt=False, **kwargs):
    cls = stringify(cls)
    return Ul(cls=(f"uk-tab{'-alt' if alt else ''}",stringify(cls)),**kwargs)(*li)

In [ ]:
# Show(Container(TabContainer(
#     Li(A("Active",href='#', cls='uk-active')),
#     Li(A("Item",href='#')),
#     Li(A("Item",href='#')),
#     Li(A("Disabled",href='#', cls='uk-disabled')),
#     alt=True)))

In [ ]:
# Show(Container(TabContainer(
#     Li(A("Active",href='#', cls='uk-active')),
#     Li(A("Item",href='#')),
#     Li(A("Item",href='#')),
#     Li(A("Disabled",href='#', cls='uk-disabled')),
#     )))

## Cards

In [ ]:
#| export
class CardT(VEnum):
    'Card styles from UIkit'
    def _generate_next_value_(name, start, count, last_values):
        return str2ukcls('card', name)
    
    default = auto()
    primary = auto()
    secondary = auto()
    danger = auto()

In [ ]:
print(enum_to_markdown_table(CardT))

| Option    | Value             |
|-----------|-------------------|
| default   | uk-card-default   |
| primary   | uk-card-primary   |
| secondary | uk-card-secondary |
| danger    | uk-card-danger    |


In [ ]:
#| export
def CardTitle(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-title',stringify(cls)), **kwargs)

def CardTitle(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-title',stringify(cls)), **kwargs)

def CardHeader(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-header',stringify(cls)), **kwargs)

def CardBody(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-body',stringify(cls)), **kwargs)

def CardFooter(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-footer',stringify(cls)), **kwargs)

def CardContainer(*c, cls=CardT.default, **kwargs):
    return fh.Div(*c, cls=('uk-card',stringify(cls)), **kwargs)

In [ ]:
#|export
def Card(*c, # Components that go in the body
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls=(), # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res.append(CardHeader(cls=header_cls)(header))
    res.append(CardBody(cls=body_cls)(*c))
    if footer: res.append(CardFooter(cls=footer_cls)(footer))
    return CardContainer(cls=cls, **kwargs)(*res)

## Tables

In [ ]:
#| export
class TableT(VEnum):
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('table', name)
    divider = auto()
    striped = auto()
    hover = auto()
    small = auto()
    large = auto()
    justify = auto()
    middle = auto()
    responsive = auto()

In [ ]:
#| export
def Table(*args, cls=(TableT.middle, TableT.divider, TableT.hover, TableT.small), **kwargs): 
    return fh.Table(cls=('uk-table', stringify(cls)), *args, **kwargs)

In [ ]:
#| export
def _TableCell(Component, *args, cls=(), shrink=False, expand=False, small=False, **kwargs):
    cls = stringify(cls)
    if shrink: cls += ' uk-table-shrink'
    if expand: cls += ' uk-table-expand'
    if small: cls += ' uk-table-small'
    return Component(*args,cls=cls, **kwargs)

@delegates(_TableCell, but=['Component'])
def Td(*args,**kwargs):  return _TableCell(fh.Td, *args, **kwargs)
@delegates(_TableCell, but=['Component'])
def Th(*args,**kwargs): return _TableCell(fh.Th, *args, **kwargs)

def Tr(*cells, cls=(), **kwargs):  return fh.Tr(*cells, cls=stringify(cls), **kwargs)
def Thead(*rows, cls=(), **kwargs): return fh.Thead(*rows, cls=stringify(cls), **kwargs)
def Tbody(*rows, cls=(), **kwargs): return fh.Tbody(*rows, cls=stringify(cls), **kwargs)

In [ ]:
#|export
def TableFromLists(header_data, body_data, footer_data=None, 
                   header_cell_render=Th,body_cell_render=Td, footer_cell_render=Td,
                   cls=(TableT.middle, TableT.divider, TableT.hover, TableT.small), **kwargs):
    
    return Table(
                Thead(Tr(*map(header_cell_render, header_data))),
                Tbody(*[Tr(*map(body_cell_render, r)) for r in body_data]),
                Tfoot(Tr(*map(footer_cell_render, footer_data))) if footer_data else '',
                cls=stringify(cls),    
                **kwargs)

In [ ]:
# Test the function
header = ['Name', 'Age', 'City']
body = [
    ['Alice', '25', 'New York'],
    ['Bob', '30', 'San Francisco'],
    ['Charlie', '35', 'London']
]
footer = ['Total', '90', '']

fh.show(TableFromLists(header, body, footer))

Name,Age,City
Alice,25,New York
Bob,30,San Francisco
Charlie,35,London
Total,90,


In [ ]:
#| export
def TableFromDicts(header_data:Sequence, body_data:Sequence[dict], footer_data=None, 
                   header_cell_render=Th, body_cell_render=lambda k,v : Td(v), footer_cell_render=lambda k,v : Td(v),
                   cls=(TableT.middle, TableT.divider, TableT.hover, TableT.small), **kwargs):
    
    return Table(
        Thead(Tr(*[header_cell_render(h) for h in header_data])),
        Tbody(*[Tr(*[body_cell_render(k, r) for k in header_data]) for r in body_data]),
        Tfoot(Tr(*[footer_cell_render(k, footer_data.get(k.lower(), '')) for k in header_data])) if footer_data else '',
        cls=stringify(cls),    
        **kwargs
    )

In [ ]:
# Example usage:
def header_render(v): return Th(v.upper())
def body_render(k, v):
    match k.lower():
        case 'name': return Td(v['name'], cls='font-bold')
        case 'age': return Td(f"{v['age']} years")
        case _: return Td(v[k.lower()])

header_data = ['Name', 'Age', 'City']
body_data = [
    {'name': 'Alice', 'age': 30, 'city': 'New York'},
    {'name': 'Bob', 'age': 25, 'city': 'London'}
]

fh.show(TableFromDicts(header_data, body_data, header_cell_render=header_render, body_cell_render=body_render))

NAME,AGE,CITY
Alice,30 years,New York
Bob,25 years,London


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
%%html
<script>
window.addEventListener('load', function() {
    for (let i = 1; i <= 6; i++) {
        document.querySelectorAll(`h${i}`).forEach(el => {
            el.classList.add(`uk-h${i}`);
        });
    }
});

document.querySelectorAll('.sidebar-item-text').forEach(el => {
  el.style.visibility = 'visible';
})

</script>